In [1]:
#Some simulation constants:
DATA_FILE = "01_clothing_data_joined_L_parts"
DATA_FOLDER = "../data/"
EXPERIMENT_CONFIG_FOLDER = "../simulation/"

# 1) Lets load the _ClothesList!

First we need to format logging to nicely format log messages as we please!

In [2]:
import logging
logger = logging.getLogger()
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter("=[%(levelname)s @ %(filename)s (L=%(lineno)s) F=%(funcName)s() ]= %(message)s")
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)   

Get the clothing data object for simulation. Because our notebooks are in a subfolder, we need to add parent folder to sys.path!

In [3]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from clothing.clothes_list import ClothesList

_ClothesList = ClothesList(DATA_FOLDER + DATA_FILE).getClothesList() # List of ClothingItem


Now we have _ClothesList, which is a list of _ClothingItem_

__ClothingItem__ contains multiple clothing parts and corresponding joints. Its main properties are:
* __.Name__
* __.getPartList()__ [._partList ] - list of _ClothingPart_
* __.getJointList()__ [._jointList] - list of _Joint_

__ClothingPart__ contains multiple points and corresponding segments. Its main properties are:
* __.Name__
* __.PointList__ - list of points (Each point is __[x,y]__ where x and y are coordinates in a plane, normalized so that both are smallest non-negative numbers able to hold the whole clothing part.
* __.getSegmentList()__ [._segmentList] - list of segments (Each segment is __[n1, n2]__ where n is the index of point in _.PointList_).

__Joint__ contains a pair of parts and a corresponding pair of segments, which together make a sewn joint. Its main properties are:
* __.getName()__ - Returns a human readable joint name including its inversion state and segment/part pairs
* __.getParts()__ [._parts] - list of two part indexes __[p1, p2]__ in the _.getPartList()_ of parent _ClothingItem_. 
* __.getSegments()__ [._segments] - list of two segment indexes __[s1, s2]__ in the _.getSegmentList()_ of the relative _ClothingPart_ from the _.getParts()_ of this Joint
* __.isInverted()__ [._inverted] - boolean - if __false__, both segments relate from lowest to highest point index, if __true__, then highest index from one matches the lowest on the other.

# Imports

In [4]:
from shapely.geometry import Point, LinearRing, LineString, MultiLineString, Polygon, GeometryCollection 
from shapely import affinity
%matplotlib inline

In [5]:
from visualizer import Visualizer
viz = Visualizer()


In [6]:
from tessellator import Tessellator

# Work with experiment generator!

In [7]:

from generator import ClothingExperimentGenerator



In [9]:
## Generates actual experiment configs
## TODO - Clothing experiment generator is not finished - need to cover all interesting cases, 
## and here potentially split in mupltiple files

gen = ClothingExperimentGenerator(_ClothesList) # agrak te bija otrs parametrs>>??, 45)
iteration = 0

current_tess = None

file = None

for gen_res in gen.generate_configuration():
    iteration = iteration + 1
    if iteration < 37401:
        continue
    if gen_res.tessellation_algorithm != current_tess:
        if(file!=None):
            file.close()
        current_tess = gen_res.tessellation_algorithm
        filename = EXPERIMENT_CONFIG_FOLDER+current_tess+"_config.json"
        file = open(filename,"w")         
    
    
    
    #if iteration%10 is 0:
    file.write(gen_res.serialize()+"\n") 
        #viz.visualize_a_clothe(a_clothe, tesselations = execute_experiment(gen_res), scale=1.05)

    
    #if iteration>5000:
    #    break
if(file!=None):
    file.close()